In [1]:
import string
import os, random, sys, time
from urllib.parse import urlparse
from selenium import webdriver
import pandas as pd
import numpy as np
#nltk.download('stopwords')

from time import time
import unicodecsv as csv
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [2]:
df1 = pd.DataFrame(pd.read_excel('TJT_datasets.xlsx'))

In [3]:
df1.head()

,JID,Jcompany,Jposition,Jdesc,Jlocation,Jfunction,Jkeywords,Jtkey
0,12,Freshworks,Senior Product Manager,Freshworks Inc. is the parent company behind t...,"Chennai, India",Product Management,"Customer-facing teams, leadership team,owner",NaN
1,19,ServiceNow,Senior Product Manager- Analytics,Company\n\nWork matters. It’s where we spend a...,"Hyderabad, IN",Product Management,"Performance Analytics, Reporting team",NaN


In [4]:
df = pd.DataFrame(index=[0], columns=df1.columns)
df

,JID,Jcompany,Jposition,Jdesc,Jlocation,Jfunction,Jkeywords,Jtkey
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
jobid = int(input("Enter Job ID:"))
for i in range(len(df1)):
    if jobid == df1['JID'][i]:
        for j in range(8):
            df[df1.columns[j]][0] = df1[df1.columns[j]][i]
df     

Enter Job ID:19


,JID,Jcompany,Jposition,Jdesc,Jlocation,Jfunction,Jkeywords,Jtkey
0,19,ServiceNow,Senior Product Manager- Analytics,Company\n\nWork matters. It’s where we spend a...,"Hyderabad, IN",Product Management,"Performance Analytics, Reporting team",NaN


In [62]:
desc = df['Jdesc'][0]
keyword = df['Jkeywords'][0]
function = df['Jfunction'][0]
desc

'Company\n\nWork matters. It’s where we spend a third of our lives. And the workplace of the future is going to be a great place. We’re dedicated to bringing that to life for people everywhere. That’s why we put people at the heart of everything we do.\n\nPeople matter. Our people have a passion for learning, building, and innovating. Whether you’re an engineer, a sales professional, a finance professional, or anything in-between, our roles aim to provide each person with meaningful impact and plenty of space to grow.\n\nRole\n\nAre you passionate about product management and relentless when it comes to delivering great outcomes for your customers? Do you sweat the details and thrive in a fast-paced, innovative environment? ServiceNow is transforming the world of work via our digital workflows and enterprise cloud platform. We are looking for an experienced Product Manager to join our Performance Analytics and Reporting team.\n\nYour technical acumen, customer empathy and data analysis

In [63]:
def obtain_features():
    # import stop words set from NLTK package
    stop_word_set = set(stopwords.words('english'))
    with open('vocab.txt',str('rb')) as source:
        rdr = csv.reader(source, encoding='utf-8')
        features = []
        skill_set = set()
        for row in rdr:
            skill = row[0].lower()
            if skill not in skill_set and skill not in stop_word_set:
                skill_set.add(skill)
                features.append(skill)
    return features

In [64]:
vocab = obtain_features()

In [65]:
temp_skills = []
def NMF_cluster(data, min_n, max_n, vocab):
    # n_features = len(vocab)
    n_features = 1000
    # Use tf-idf features for NMF.
    print("\nExtracting tf-idf features for NMF...")
    tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1,
                                       #vocabulary=vocab,
                                       stop_words='english',
                                       #stop_words=None,
                                       token_pattern=r"(?u)\S+",
                                       max_features=n_features, sublinear_tf=True,
                                       ngram_range=(min_n, max_n))
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(data)
    # print out features generated by tf-idf vectorizer
    feature = tfidf_vectorizer.get_feature_names()
    for each in feature:
        temp_skills.append(each)
      
    print('# of features: %d'%(len(tfidf_vectorizer.get_feature_names())))

In [66]:
NMF_cluster([desc], 1, 3, vocab)
temp_skills


Extracting tf-idf features for NMF...
# of features: 670


['20%',
 '20% annually',
 '7+',
 '7+ years',
 '7+ years experience',
 'ability',
 'ability learn',
 'ability learn new',
 'ability make',
 'ability make trade-off',
 'ability relate',
 'ability relate various',
 'activities,',
 'activities, i.e.,',
 'activities, i.e., good',
 'acumen,',
 'acumen, customer',
 'acumen, customer empathy',
 'administration',
 'administration experiences.',
 'administration experiences. role',
 'agile',
 'agile product',
 'agile product management',
 'aim',
 'aim provide',
 'aim provide person',
 'ambiguity',
 'ambiguity enjoys',
 'ambiguity enjoys working',
 'analysis',
 'analysis skills',
 'analysis skills help',
 'analytics',
 'analytics reporting',
 'analytics reporting serve',
 'analytics reporting team.',
 'annually',
 'bringing',
 'bringing life',
 'bringing life people',
 'brings',
 'brings energy,',
 'brings energy, curiosity,',
 'building,',
 'building, innovating.',
 'building, innovating. you’re',
 'business',
 'business perspective',
 'business

In [67]:
def filter_skills(temp_skills):
    
    threewordskills = [word for word in temp_skills if len(word.split(' ')) > 2]
    
    nopuncskills = [word for word in threewordskills if ',' not in word and '.' not in word and '+' not in word]

    unrelatedwordsskills = [word for word in nopuncskills if 'like' not in word]

    sortedbyvocab = []
    for word in unrelatedwordsskills:
        for i in vocab:
            if i in word and word not in sortedbyvocab:
                sortedbyvocab.append(word)
    
    keywords = keyword.split(',')
    functions = function.split(' ')
    keywords.extend(functions)
    
    skills = [word for word in sortedbyvocab if 'skills' in word.split()[2]]
    for word in sortedbyvocab:
        for i in keywords:
            if i.lower() in word and word not in skills:
                skills.append(word)
                keywords.remove(i)
    return skills

In [68]:
skills = filter_skills(temp_skills)
skills

['data analysis skills',
 'exceptional presentation skills',
 'agile product management',
 'join performance analytics',
 'leading service management']

In [ ]:
step2a_df = pd.DataFrame(index=[0],columns=['Company Name', 'Job Location', 'Headquarter', 'Company Details', 'Job Title', 'Roles', 'Skills', 'PHM'])
step2a_df

In [ ]:
for i in df:
    if 'company' in i:
        step2a_df['Company Name'][0] = df[i][0]
    if 'location' in i:
        step2a_df['Job Location'][0] = df[i][0]
    if 'role' in i or 'function' in i:
        step2a_df['Roles'][0] = df[i][0]
    if 'position' in i or 'title' in i:
        step2a_df['Job Title'][0] = df[i][0]
    if 'skills' in i:
        step2a_df['Skills'][0] = df[i][0]
    #if 'desc' in i or 'description' in i:
        #step2a_df['Skills'][0] = df[i][0]

In [ ]:
step2a_df

## Web Scrapping

In [ ]:
#import ipywidgets as widgets
#company_name = widgets.Dropdown(step2a_df['Company Name'])

In [ ]:
browser = webdriver.Chrome('driver/chromedriver.exe')

In [ ]:
browser.get('https://www.linkedin.com/uas/login')

In [ ]:
file = open('config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

In [ ]:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

In [ ]:
elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

In [ ]:
elementID.submit()

In [ ]:
company_name = df['Jcompany']
#company_name='Freshworks'
fullLink = 'https://www.linkedin.com/search/results/companies/?keywords='+company_name
browser.get(fullLink)

In [ ]:
element = browser.find_element_by_link_text(company_name) 
element.click() 
time.sleep(5)

In [ ]:
browser.execute_script("window.scrollTo(0, 500)") 
element = browser.find_element_by_link_text("See all") 
element.click() 

# Company Details

In [ ]:
time.sleep(5)
company_details = browser.find_element_by_class_name('break-words').text
step2a_df['Company Details']= company_details

# Company Headquarters

In [ ]:
company_headquarters = browser.find_element_by_class_name('mb3').find_element_by_class_name('overflow-hidden').find_element_by_xpath('(//dd[5])').text
step2a_df['Headquarter']=company_headquarters

In [ ]:
PHM = pd.DataFrame(pd.read_csv('C:/Users/Divya/PHM.csv'))
for i in range(len(PHM['Interviewee'])):
    if PHM['Interviewee'][i] == step2a_df['Job Title'][0]:
        step2a_df['PHM'][0]=PHM['Hiring Manager'][i]
step2a_df

# Potential Hiring Manager

In [ ]:
def check_empty(data):
    if data == '':
        return ''
    else:
        return data + '%2C'

In [ ]:
PHM = step2a_df['PHM'][0]
location = step2a_df['Job Location'][0]
domain = step2a_df['Roles'][0]
temp_url = check_empty(PHM) + check_empty(location) + check_empty(domain)
fullLink = browser.current_url[:-6] + '/people/?keywords=' +  temp_url[:-3]
browser.get(fullLink)

In [ ]:
PHM_df = pd.DataFrame(index=[i for i in range(10)],columns=['Employee Name', 'Job Title', 'Job Location', 'About'])
PHM_df.head()

In [ ]:
current_url = browser.current_url
for i in range(1,14):
    try:    
        if i > 12:
            browser.execute_script("window.scrollTo(0, 1500)")
            time.sleep(3)
        browser.find_element_by_class_name('org-people-profiles-module__profile-list').find_element_by_xpath('(//li[@class="org-people-profiles-module__profile-item"]['+str(i)+'])').find_element_by_class_name('org-people-profile-card__profile-info').click()
        time.sleep(3)
        PHM_df['Employee Name'][i-1] = browser.find_element_by_class_name('pv-top-card--list').find_element_by_tag_name('li').text
        PHM_df['Job Title'][i-1] = browser.find_element_by_class_name('ph5').find_element_by_class_name('mt2').find_element_by_class_name('mt1').text
        loc = browser.find_element_by_class_name('pv-top-card--list-bullet').find_element_by_tag_name('li').text
        if "connection" in loc:
            PHM_df['Job Location'][i-1] = np.nan
        else:
            PHM_df['Job Location'][i-1] = loc
        PHM_df['About'][i-1] = browser.find_element_by_class_name('pv-about__summary-text').text
    except:
        pass
    browser.get(current_url)



In [ ]:
PHM_df

In [ ]:
PHM_df=PHM_df.dropna(how='all')
PHM_df = PHM_df.reset_index(drop=True)

In [ ]:
scores_df = pd.DataFrame({'scores':[0] * len(PHM_df)})
scores_df.head()

In [ ]:
for i in range(len(PHM_df)):
    #Scoring based on About Section
    if str(np.nan) in str(PHM_df['About'][i]):
        scores_df['scores'][i] = 1
    elif 'hiring' or 'hire' or 'recruiting' or 'recruitment' in str(PHM_df['About'][i]):
        scores_df['scores'][i] = 3
    else:
        scores_df['scores'][i] = 2
        
    #Scoring based on Location 
    if str(np.nan) in str(PHM_df['Job Location'][i]):
        scores_df['scores'][i] += 1
    elif 'chennai' in str(PHM_df['Job Location'][i]).lower(): #take chennai from dataframe on employee use split(',')[0] to take only chennai
        scores_df['scores'][i] += 3
    else:
        scores_df['scores'][i] += 2
    
    #Scoring based on domain
    if str(np.nan) in str(PHM_df['Job Title'][i]):
        scores_df['scores'][i] += 1
    elif 'Product Management' in str(PHM_df['Job Title'][i]): #take prodcut management from dataframe on employee
        scores_df['scores'][i] += 3
    else:
        scores_df['scores'][i] += 2
    

In [ ]:
scores_df


In [ ]:
scores_df.sort_values(by=['scores'], inplace=True, ascending=False)
scores_df

In [ ]:
#Top 3 PHM's
top3 = scores_df.index.values[:3]
top3_df = pd.DataFrame(columns=['Employee Name', 'Job Title', 'Job Location', 'About'])
for i in range(len(top3)):
    top3_df  = top3_df.append(PHM_df.iloc[top3[i]])
top3_df = top3_df.reset_index(drop=True)
top3_df